(sec:deep-q-learning)=
# 深層強化学習

前節{ref}`sec:q-learning`ではTD学習を対象としてSARSAとQ学習について紹介した。

しかし、前節で解説したこれらの手法は**Qテーブルを離散化しなければならない**という欠点を持つ。

CartPoleの例では、浮動小数のパラメータ4つを8段階に量子化したために、状態空間の数は4096個であり、出力の操作の種類は右に動くか、左に動くかの2つであった。従って、Qテーブルのサイズは4096×2となる。

しかし、このテーブルのサイズは、パラメータや出力の数が増えたり、パラメータをより細かく離散化したりすると、急激にテーブルのサイズが増え、学習に時間がかかるだけでなく、そもそも状態空間の広さから学習が難しくなる、という問題があった。

そんな時にDeepMindの研究者らのチームによって公開された論文が「Playing Atari with Deep Reinforcement Learning」({cite}`mnih2013playing`)である。

そもそもニューラルネットワークは入出力がともに多次元の複雑な関数を表す能力に優れており、この論文ではニューラルネットによって、価値行動関数 $Q(s, a)$ を表現させている。このようなニューラルネットを**Qネットワーク**と呼ぶ。

In [12]:
"""
Google Colabの準備
"""

IN_COLAB = True
try:
    import google.colab

    print("You are running the code in Google Colab.")
except ImportError:
    IN_COLAB = False
    print("You are running the code on the local computer.")

if IN_COLAB:
    # Gymnasiumのインストール
    !pip install "gymnasium[classic-control]"
    pass

You are running the code on the local computer.


In [ ]:
import random
from collections import deque

import numpy as np
import seaborn as sns
import IPython.display as display
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from matplotlib.animation import ArtistAnimation

try:
    from myst_nb import glue
except ImportError:
    glue = lambda *args, **kwargs: None

# パラメータ
n_episodes = 100
glue("n_episodes", n_episodes)

# 乱数のシードを固定
random.seed(31415)
np.random.seed(31415)

# グラフの設定
rc = {
    "figure.dpi": 150,
    "axes.linewidth": 1,
    "axes.edgecolor": "black",
    "grid.color": "gray",
    "grid.linestyle": "--",
    "grid.linewidth": 0.5,
    "xtick.major.size": 2,
    "ytick.major.size": 2,
    "legend.frameon": True,
    "legend.borderpad": 0.5,
    "legend.facecolor": "white",
    "legend.edgecolor": "black",
    "legend.framealpha": 1.0,
}
sns.set_theme(style="whitegrid", palette="colorblind", rc=rc)

100

## 深層Q学習の概要

実は、DeepMind社の深層Q学習の論文以前にも、ニューラルネットワークを用いて強化学習をしよう、という試み自体は存在していた。

それらの手法は、シミュレーション環境から状態パラメータを受け取り、それを学習用データセットとしてためておいて、価値行動関数を表すQネットワークを訓練するというもので、この考え方は深層Q学習にも共通している。

これに対し、深層Q学習の論文では、

1. 状態パラメータを受け取らず、画像を入力としてプレイを行う
2. 経験リプレイを使うことで、効率的にネットワークを学習する

という2点が新しく提案されている。

深層Q学習の論文では、Atariゲーム (ATARI社が過去に開発したビデオゲーム)を題材としており、これらは所謂普通のビデオゲームであるため、状態パラメータを受け取ることはできず、**画像だけから、どのようなプレイを行なうかを判断しなければならない**。この点で、状態パラメータを受け取ることができる前節のCartPole環境より難しいタスクである。

また、状態パラメータが入力の場合も、画像が入力の場合も、時系列的に連続したデータから、ニューラルネットワークの訓練に用いるミニバッチを構成すると、勾配に強いバイアスがかかり、学習が進みづらくなるという問題がある。本論文では、**リプレイバッファと呼ばれる、過去の状態を記録しておくメモリを用意**しておき、その中からランダムに状態をサンプリングすることで、確率的最急降下法を効率化している。

以下では、まず状態変数をネットワークに入力する実装を紹介した後、画像だけを入力としてプレイを行なうAIへと改変する。

## 下準備

深層Q学習のコア部分を紹介する前に、いくつか下準備を行なっておく。まずは、PyTorchをインポートして、単純なニューラルネットワークを定義しておく。

なお、Q学習において、状態価値関数は任意の実数を取って良いので、最終層の活性化関数は不要である。

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F


class Network(nn.Sequential):
    """
    シンプルなmulti-layer perceptron
    """

    def __init__(self, n_inputs, n_outputs):
        super(Network, self).__init__(
            nn.Linear(n_inputs, 128, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Linear(64, n_outputs),
        )

In [ ]:
# デバイスの設定
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Your device is {device} ({torch.cuda.get_device_name(device)}).")
else:
    device = torch.device("cpu")
    print(f"Your device is {device}.")

# ネットワークの初期化
# CartPoleは状態変数の数が4で、出力パラメータ数が2
q_net_online = Network(4, 2)
q_net_target = Network(4, 2)
q_net_online.to(device)
q_net_target.to(device)

# オプティマイザの初期化
optim = torch.optim.Adam(q_net_online.parameters(), lr=1.0e-3)

Your device is cuda (NVIDIA GeForce RTX 3080 Ti).


In [ ]:
class ReplayMemoryDataset(torch.utils.data.Dataset):
    def __init__(self, memory):
        self.memory = memory

    def __len__(self):
        return len(self.memory)

    def __getitem__(self, idx):
        m = self.memory[idx]
        return m

この他、Gymnasiumの初期化やパラメータの設定は以下のように設定する。

In [ ]:
import gymnasium as gym

# Q学習のパラメータ
gamma = 0.99

# 深層Q学習のパラメータ
batch_size = 32
steps_per_episode = 1000
memory_size = 10000

# ゲーム環境の作成
env = gym.make("CartPole-v1", render_mode="rgb_array")

In [ ]:
glue("batch_size", batch_size)
glue("steps_per_episode", steps_per_episode)
glue("memory_size", memory_size)

32

1000

10000

今回の実験では、1プレイ (エピソード)ごとにサイズが{glue:}`batch_size`のミニバッチで{glue:}`steps_per_episode`ステップ分訓練を行う。

過去のゲームの状態を保存するリプレイメモリのサイズは最大{glue:}`memory_size`状態としておき、それ以後は古いものから順に捨てていくこととする。このようなリプレイ・バッファの実装は`collections.deque`を用いると容易である。

```python
from collections import deque

replay_buffer = deque(maxlen=memory_size)
```

In [ ]:
# リプレイ・バッファの準備
replay_buffer = deque(maxlen=memory_size)

# 深層Q学習では, 線形にεを減少させる
e0 = 0.5
e1 = 0.005
epsilons = np.linspace(e0, e1, n_episodes)

# エピソードのループ
pbar = tqdm(total=n_episodes * steps_per_episode)
for epi in range(n_episodes):
    # ゲーム環境のリセット
    s0, _ = env.reset()
    eps = epsilons[epi]

    # エピソード開始
    while True:
        # Q-networkを使って行動を選択
        inputs = torch.Tensor(s0)
        inputs = inputs.unsqueeze(0).float().to(device)

        # ε-greedy法
        if np.random.rand() < eps:
            a0 = env.action_space.sample()
        else:
            with torch.no_grad():
                q_net_online.eval()
                q_values = q_net_online(inputs)

            q_values = q_values.detach().squeeze().cpu().numpy()
            a0 = np.argmax(q_values)

        # 行動の選択
        s1, reward, done, _, _ = env.step(a0)

        # リプレイメモリに記録
        replay_buffer.append((s0, a0, reward, s1, done))

        # 次の状態に遷移
        s0 = s1

        if done:
            break

    # データセットの用意
    memory_dataset = ReplayMemoryDataset(replay_buffer)
    memory_sampler = torch.utils.data.RandomSampler(
        replay_buffer,
        replacement=True,
        num_samples=batch_size * steps_per_episode,
    )
    memory_loader = torch.utils.data.DataLoader(
        memory_dataset,
        batch_size=batch_size,
        sampler=memory_sampler,
    )

    # 学習ループ
    q_net_online.train()
    for i, memory in enumerate(memory_loader):
        s0, a0, reward, s1, done = memory

        s0 = s0.float().to(device)
        a0 = a0.long().to(device)
        reward = reward.float().to(device)
        s1 = s1.float().to(device)
        done = done.float().to(device)

        q_values = q_net_online(s0)
        q0 = torch.gather(q_values, 1, a0.unsqueeze(1)).squeeze(-1)

        with torch.no_grad():
            q_net_target.eval()
            q1 = q_net_target(s1)
            q_max = torch.max(q1, dim=1)[0]

        loss = F.smooth_l1_loss(q0, reward + gamma * q_max * (1 - done))

        optim.zero_grad()
        loss.backward()
        optim.step()

        if i % 100 == 0:
            pbar.set_description(f"Episode {epi+1}/{n_episodes}, Loss: {loss.item():.3f}")

        pbar.update()

    # Q-networkの更新
    if (epi + 1) % 5 == 0:
        q_net_target.load_state_dict(q_net_online.state_dict())

In [ ]:
frames = []
obsrv, _ = env.reset()
while True:
    img = env.render()
    frames.append(img)

    # Q-networkを使ってQ値を計算
    inputs = torch.Tensor(obsrv)
    inputs = inputs.unsqueeze(0).float().to(device)
    with torch.no_grad():
        q_net_online.eval()
        q_values = q_net_online(inputs).detach().squeeze().cpu().numpy()

    # Q値が最大となる行動を選択
    a = np.argmax(q_values)

    obsrv, reward, done, _, _ = env.step(a)
    if done:
        break

In [ ]:
# アニメーションの描画
fig, ax = plt.subplots(dpi=100)
ax.set(xticks=[], yticks=[])

# 各フレームの描画
draw = []
for i, f in enumerate(frames):
    ims = plt.imshow(f)
    txt = plt.text(20, 30, f"frame #{i+1:d}")
    draw.append([ims, txt])
    fig.tight_layout()

# アニメーションの作成
ani = ArtistAnimation(fig, draw, interval=100, blit=True)
html = display.HTML(ani.to_jshtml())
display.display(html)

# Matplotlibのウィンドウを閉じる
plt.close()

## 参考文献

```{bibliography}
:filter: docname in docnames
```